# Database Setup

This notebook sets up the SQLite database (`labour_rules.db`) used to store and query contract rules.

**Steps:**
1. **Schema Definition**: Creates the `chunks` table with fields for file info, text, and classification levels.
2. **Data Loading**: Reads the classification JSON produced in the previous step and inserts it into the database.
3. **Verification**: Queries the database to ensure `LABOUR` chunks are correctly indexed.


In [ ]:
import sqlite3
import json
from pathlib import Path

# ==================== CONFIGURATION ====================
DB_PATH = "labour_rules.db"
CHUNKS_DIR = Path("/Users/swathi.gnanasekar/Documents/Vista_Vu_Project/Phase 1/Docling_Tweak/CHUNK_NEW")  # Your folder with .txt files
CLASSIFICATIONS_JSON = "/Users/swathi.gnanasekar/Documents/Vista_Vu_Project/Phase 1/Docling_Tweak/Categorized/result_version 0.json"  # Your JSON file with all categories

# ==================== CREATE DATABASE ====================
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS chunks (
    chunk_id INTEGER PRIMARY KEY AUTOINCREMENT,
    chunk_file_name VARCHAR(255) UNIQUE NOT NULL,
    chunk_text TEXT NOT NULL,
    contract_section_type VARCHAR(50) NOT NULL,
    second_level VARCHAR(50) NOT NULL,
    third_level VARCHAR(50) NOT NULL,
    reasoning TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Create index
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_third_level ON chunks(third_level)
""")

conn.commit()
print("✅ Database created: labour_rules.db")

# ==================== LOAD DATA ====================
# Read classifications JSON
with open(CLASSIFICATIONS_JSON, 'r') as f:
    classifications_data = json.load(f)

print(f"\n📂 Found {len(classifications_data)} chunks in JSON")

# Load each chunk
loaded_count = 0
for item in classifications_data:
    chunk_file_name = item['chunk_file']
    classification = item['classification']
    
    # Find and read the text file
    # Adjust path based on your folder structure
    chunk_path = CHUNKS_DIR / chunk_file_name
    
    if not chunk_path.exists():
        print(f"⚠️  File not found: {chunk_path}")
        continue
    
    # Read chunk text
    with open(chunk_path, 'r', encoding='utf-8') as f:
        chunk_text = f.read()
    
    # Insert into database
    try:
        cursor.execute("""
            INSERT INTO chunks 
            (chunk_file_name, chunk_text, contract_section_type, second_level, third_level, reasoning)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            chunk_file_name,
            chunk_text,
            classification['Contract Section Type'],
            classification['Second Level'],
            classification['Third Level'],
            classification['Reasoning']
        ))
        loaded_count += 1
    except sqlite3.IntegrityError:
        print(f"⚠️  Duplicate: {chunk_file_name}")

conn.commit()
print(f"\n✅ Loaded {loaded_count} chunks into database")

# ==================== VERIFY LABOUR CHUNKS ====================
cursor.execute("SELECT COUNT(*) FROM chunks WHERE third_level = 'LABOUR'")
labour_count = cursor.fetchone()[0]
print(f"✅ Found {labour_count} LABOUR chunks")

# Show sample
cursor.execute("""
    SELECT chunk_file_name, contract_section_type, third_level 
    FROM chunks 
    WHERE third_level = 'LABOUR' 
    LIMIT 5
""")
print("\n📋 Sample LABOUR chunks:")
for row in cursor.fetchall():
    print(f"  • {row[0]} ({row[1]})")

conn.close()
print("\n🎉 Database setup complete!")

✅ Database created: labour_rules.db

📂 Found 155 chunks in JSON
⚠️  Duplicate: Boilermakers Collective-Agreement/01__ARTICLE_100_-PURPOSE.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/02__ARTICLE_200_-RECOGNITION_AND_CRAFT_JURISDICTION.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/03__ARTICLE_300_-MANAGEMENT_RIGHTS.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/04__ARTICLE_400_-UNION_SECURITY_AND_DUES_COLLECTION.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/05__ARTICLE_500_-NO_STRIKES_OR_LOCKOUTS.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/06__ARTICLE_600_-WORKING_CONDITIONS_SAFEFTY_MEASURES_HEALTH_AND_SANITATION.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/07__ARTICLE_700_-WELDING_TESTS.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/08__ARTICLE_800_-ACCESS_TO_JOBS.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/09__ARTICLE_900_-JOB_STEWARDS.txt
⚠️  Duplicate: Boilermakers Collective-Agreement/10__ARTICLE_1000_-GRIEVANCE_PROCEDURE.

In [3]:
import sqlite3

def get_labour_chunks():
    """Fetch all LABOUR chunks for processing"""
    conn = sqlite3.connect("labour_rules.db")
    cursor = conn.cursor()
    
    cursor.execute("""
        SELECT 
            chunk_id,
            chunk_file_name,
            chunk_text,
            contract_section_type,
            second_level,
            reasoning
        FROM chunks
        WHERE third_level = 'LABOUR'
        ORDER BY chunk_file_name
    """)
    
    chunks = cursor.fetchall()
    conn.close()
    
    return chunks

# Usage
labour_chunks = get_labour_chunks()
print(f"Found {len(labour_chunks)} LABOUR chunks\n")

# Print first chunk
if labour_chunks:
    chunk_id, file_name, text, section_type, second_level, reasoning = labour_chunks[0]
    print(f"Chunk ID: {chunk_id}")
    print(f"File: {file_name}")
    print(f"Type: {section_type} -> {second_level} -> LABOUR")
    print(f"\nText preview:\n{text[:300]}...")

Found 42 LABOUR chunks

Chunk ID: 7
File: Boilermakers Collective-Agreement/07__ARTICLE_700_-WELDING_TESTS.txt
Type: COMPENSATION -> PAYABLE_ENTITLEMENT -> LABOUR

Text preview:
## ARTICLE 7.00 -WELDING TESTS

## 7.01

Any welder who is required to do additional testing by the Employer, shall be paid for weld time of four (4) hours per successful test (excluding the boiler ¼ Down Hand Plate or other single ¼ plate test, will be paid two (2) hours). Additional test consists ...
